<a href="https://colab.research.google.com/github/DinhTienTai/Causal-model/blob/master/Intro_dowhy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Model with 3 backdoor confounders, 2 instrumental variables and 2 effect modifiers

In [0]:
! git clone https://github.com/microsoft/dowhy.git

In [0]:
! pip install --upgrade https://github.com/microsoft/dowhy/tarball/master

In [0]:
import dowhy
dowhy.__version__

In [0]:
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import logging

In [0]:
from dowhy import CausalModel
import dowhy.datasets

In [0]:
!pip install econml

In [0]:
import econml

Prepare data for practice

In [0]:
data = dowhy.datasets.linear_dataset(beta = 5,
                                     num_common_causes = 3,
                                     num_samples = 1000,
                                     num_effect_modifiers=2,
                                     num_instruments = 2,
                                     num_treatments = 1,
                                     treatment_is_binary = False)

In [0]:
# Object data is an dictionany with the following keys
data.keys()

In [0]:
# Data is stored in "df"
 data["df"]

In [0]:
# Seeing some variables
data["treatment_name"]

In [0]:
data["outcome_name"]

In [0]:
data["common_causes_names"]

In [0]:
data["instrument_names"]

In [0]:
data["effect_modifier_names"]

In [0]:
# Pairplot
import seaborn as sns

In [0]:
gpair = sns.pairplot(data["df"], diag_kind = "kde", kind = "reg", markers = ".",
              plot_kws = {"color": "purple", "scatter_kws":{"color": "red", "alpha":0.05}},
              diag_kws = {"color":"orangered", "shade":True})

In [0]:
# Default plot
data["dot_graph"]

In [0]:
data["gml_graph"]

### Step 1: Set up causal model

In [0]:
model = CausalModel(data=data["df"], 
                    treatment=data["treatment_name"], 
                    outcome=data["outcome_name"],
                    graph=data["gml_graph"])

In [0]:
model.view_model()

### Identifying estimand (predictors need to be estimated)

In [0]:
identified_estimand= model.identify_effect(proceed_when_unidentifiable=True)
print(identified_estimand)

In [0]:
identified_estimand.estimands

In [0]:
identified_estimand.estimand_type

In [0]:
identified_estimand.backdoor_variables

### Estimate averaged treatment effect

In [0]:
# Linear regression
linear_estimate = model.estimate_effect(identified_estimand, 
                                        method_name="backdoor.linear_regression",
                                        control_value=0,
                                        treatment_value=1,
                                        test_significance=True,
                                        target_units='ate',
                                       )

print(linear_estimate)

In [0]:
## With method: Double Machine learning CATE Estimator
from sklearn.preprocessing import PolynomialFeatures as PolyF
from sklearn.linear_model import ElasticNetCV as EnetCV
from sklearn.ensemble import RandomForestRegressor as RFReg

dml_estimate = model.estimate_effect(identified_estimand,
                                     method_name="backdoor.econml.dml.DMLCateEstimator",
                                     confidence_intervals=True,
                                     method_params={"init_params":{'model_y': RFReg(),
                                                                   'model_t': RFReg(),
                                                              "model_final": EnetCV(), 
                                                              'featurizer':PolyF(degree=1, include_bias=True),},
                                               "fit_params":{'inference': 'bootstrap',}
                                                   }
                                    )
print(dml_estimate)

In [0]:
"Treatment effect is {}".format(np.mean(dml_estimate.cate_estimates))

In [0]:
# Plotting 
gest = sns.kdeplot(dml_estimate.cate_estimates.reshape(1000,), shade =True, color = "red")

In [0]:
"True effect is {}".format(data["ate"])